In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## Stuff Documents Chain

`stuff documents chain`("stuff"는 "채우다" 또는 "채우기 위해"의 의미)는 문서 체인 중 가장 간단한 방식입니다. 문서 목록을 가져와서 모두 프롬프트에 삽입한 다음, 그 프롬프트를 LLM에 전달합니다.

이 체인은 문서가 작고 대부분의 호출에 몇 개만 전달되는 애플리케이션에 적합합니다.


![](https://js.langchain.com/assets/images/stuff-818da4c66ee17911bc8861c089316579.jpg)


`langchain_community.chat_models`, `langchain_core.prompts`, `langchain.chains.combine_documents` 모듈에서 필요한 클래스와 함수를 가져옵니다.

`ChatOpenAI`, `ChatPromptTemplate`, 그리고 `create_stuff_documents_chain` 함수를 사용하여 챗봇 프롬프트를 생성하는 과정을 보여줍니다.

사용자로부터 입력받은 문장을 요약하는 요청을 처리하기 위한 프롬프트 템플릿을 정의합니다.

이 템플릿은 시스템이 전문 요약가 역할을 수행하도록 요청하고, 사용자에게 한국어로 불릿 포인트 형식의 요약, 각 문장의 의미에 맞는 이모지 사용, 다양한 이모지를 활용하여 요약을 더 흥미롭게 만들라는 지시를 포함합니다.


In [5]:
!pip install -U langchain langchainhub langchain_openai langchain_community -q

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert summarizer. Please summarize the following sentence.",
        ),
        (
            "user",
            "Please summarize the sentence according to the following request."
            "\nREQUEST:\n"
            "1. Summarize the main points in bullet points in Korean."
            "2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence."
            "3. Use various emojis to make the summary more interesting."
            "\n\nCONTEXT: {context}\n\nSUMMARY:",
        ),
    ]
)
prompt

ChatPromptTemplate(input_variables=['context'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert summarizer. Please summarize the following sentence.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='Please summarize the sentence according to the following request.\nREQUEST:\n1. Summarize the main points in bullet points in Korean.2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.3. Use various emojis to make the summary more interesting.\n\nCONTEXT: {context}\n\nSUMMARY:'))])

(혹은 아래의 코드를 실행하여 프롬프트를 내려받습니다)


In [2]:
from langchain import hub

prompt = hub.pull("teddynote/summary-stuff-documents-korean")
prompt

PromptTemplate(input_variables=['context'], template='Please summarize the sentence according to the following REQUEST.\nREQUEST:\n1. Summarize the main points in bullet points in KOREAN.\n2. Each summarized sentence must start with an emoji that fits the meaning of the each sentence.\n3. Use various emojis to make the summary more interesting.\n4. Translate the summary into KOREAN if it is written in ENGLISH.\n5. DO NOT translate any technical terms.\n6. DO NOT include any unnecessary information.\n\nCONTEXT:\n{context}\n\nSUMMARY:"\n')

`TextLoader` 를 사용하여 뉴스기사를 로드하고, Document를 생성합니다.


`langchain_community.document_loaders` 모듈에서 `TextLoader` 클래스를 임포트하여 사용합니다. `TextLoader` 인스턴스는 특정 파일(`data/news.txt`)에서 문서를 로드하는 데 사용됩니다.


In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/news.txt")
docs = loader.load()
print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

문서의 수: 1

[메타데이터]

{'source': 'data/news.txt'}

========= [앞부분] 미리보기 =========

제목: 
AI2, 상업 활용까지 자유로운 '진짜' 오픈 소스 LLM '올모' 출시

내용:
앨런AI연구소(AI2)가 완전한 오픈 소스 대형언어모델(LLM) '올모(OLMo)’를 출시했다. 데이터 수집, 학습, 배포의 전 과정을 투명하게 공개한 데다 상업적 사용까지 허용한 진정한 의미의 오픈 소스 LLM이라는 평가다.
벤처비트는 1일(현지시간) 비영리 민간 AI 연구기관인 AI2가 ‘최초의 진정한 오픈 소스 LLM 및 프레임워크’라고 소개한 ‘올모’를 출시했다고 보도했다. 
이에 따르면 올모는 모델 코드와 모델 가중치뿐만 아니라 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷도 제공한다. 이를 통해 모델이 어떻게 구축되었는지 심층적으로 분석, LLM의 작동 방식과 응답을 생성하는 원리를 더 잘 이해할 수 있다. 
올모 프레임워크는 70억 매개변수의 ‘올모 7B’ 등 4가지 변형 모델과 10억 매개변수의 ‘올모 1B’ 모델을 제공한다. 모델들은 훈련 데이터를 생성하는 코드를 포함해 


`MyCallbackHandler` 클래스는 `BaseCallbackHandler`를 상속받아, 언어 모델이 새로운 토큰을 생성할 때마다 해당 토큰을 출력하는 기능을 추가합니다.

`ChatOpenAI` 객체는 GPT-3.5-turbo 모델을 사용하며, 스트리밍 모드와 낮은 온도 설정을 통해 더 일관된 응답을 생성하도록 구성됩니다. 이 객체는 콜백으로 `MyCallbackHandler` 인스턴스를 사용합니다.

마지막으로, `create_stuff_documents_chain` 함수를 사용하여 문서 생성 체인을 만들고, 이 체인을 통해 주어진 문맥(`docs`)에 대한 응답(`answer`)을 생성합니다.


In [5]:
from langchain.callbacks.base import BaseCallbackHandler


class MyCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(f"{token}", end="", flush=True)


llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    temperature=0.01,
    callbacks=[MyCallbackHandler()],
)
chain = create_stuff_documents_chain(llm, prompt)
answer = chain.invoke({"context": docs})

🚀 앨런AI연구소(AI2)가 '올모(OLMo)'라는 오픈 소스 대형언어모델(LLM)을 출시했다.
🔍 올모는 데이터 수집, 학습, 배포의 전 과정을 투명하게 공개하고 상업적 사용까지 허용하는 진정한 의미의 오픈 소스 LLM이다.
🔧 올모는 모델 코드, 모델 가중치, 훈련 코드, 훈련 데이터, 관련 툴킷 및 평가 툴킷을 제공한다.
📚 올모 프레임워크에는 '올모 7B'와 '올모 1B' 등 4가지 변형 모델이 포함되어 있다.
💡 올모는 AI2의 '돌마(Dolma)' 데이터 세트를 기반으로 구축되었으며, 상업적 활용에 제한이 없는 아파치 2.0 라이선스를 따른다.
🌍 올모는 다양한 언어 처리 벤치마크 테스트에서 상업용 제품과 동등한 성능을 보여준다.
🔧 AI2는 올모를 계속해서 향상시킬 예정이며, 올모의 출시는 AI 모델에 대한 더 깊은 이해를 위한 기반을 구축하는 것이다.
👨‍💻 얀 르쿤 메타 수석 과학자는 오픈 소스 기반 모델이 AI의 혁신과 개발을 촉진하는 데 매우 중요하다고 말했다.
🔗 올모에 대한 모든 리소스는 깃허브 및 허깅페이스에서 무료로 사용할 수 있다.